<a href="https://colab.research.google.com/github/BHUVAN-005/RECOMMENDATION-SYSTEM/blob/main/RECOMMENDATION_SYSTEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommendation System using SVD (Collaborative Filtering)
# Dataset: MovieLens 100K
# Done By: K Naga Santosh Sai Bhuvan


# Step 1: Install and Import Required Libraries

In [1]:
!pip install surprise
!pip install numpy==1.24.4
!pip install scikit-surprise

In [1]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split, cross_validate
from surprise import accuracy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Step 2: Load MovieLens Dataset (Built-in)

In [2]:
data = Dataset.load_builtin('ml-100k')  # MovieLens 100K dataset
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


#  Step 3: Build SVD Collaborative Filtering Model

In [3]:
model = SVD()
model.fit(trainset)

# Step 4: Make Predictions & Evaluate

In [4]:
predictions = model.test(testset)

# Print a few predictions
print("📌 Sample Predictions (User, Item, Actual, Predicted):")
for uid, iid, true_r, est, _ in predictions[:5]:
    print(f"User {uid} -> Movie {iid}: Actual = {true_r}, Predicted = {round(est, 2)}")

📌 Sample Predictions (User, Item, Actual, Predicted):
User 907 -> Movie 143: Actual = 5.0, Predicted = 5
User 371 -> Movie 210: Actual = 4.0, Predicted = 4.33
User 218 -> Movie 42: Actual = 4.0, Predicted = 3.44
User 829 -> Movie 170: Actual = 4.0, Predicted = 3.91
User 733 -> Movie 277: Actual = 1.0, Predicted = 3.05


#  Step 5: Evaluate with RMSE and MAE

In [5]:
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

RMSE: 0.9356
MAE:  0.7371


#  Step 6: Cross Validation

In [6]:
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9337  0.9424  0.9257  0.9392  0.9403  0.9363  0.0060  
MAE (testset)     0.7358  0.7418  0.7284  0.7418  0.7401  0.7376  0.0051  
Fit time          1.17    1.21    1.14    1.18    1.37    1.22    0.08    
Test time         0.25    0.10    0.15    0.26    0.23    0.20    0.06    


{'test_rmse': array([0.93371679, 0.94244697, 0.92573118, 0.93922498, 0.94033165]),
 'test_mae': array([0.73577338, 0.7417537 , 0.72842084, 0.74179102, 0.7400922 ]),
 'fit_time': (1.1734559535980225,
  1.2120287418365479,
  1.1435184478759766,
  1.179847240447998,
  1.3678562641143799),
 'test_time': (0.25493860244750977,
  0.10095643997192383,
  0.14827251434326172,
  0.2564980983734131,
  0.22832942008972168)}

#  Step 7: Generate Top-N Recommendations for a User

In [7]:
from collections import defaultdict

def get_top_n(predictions, n=5):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Sort predictions for each user and retrieve top-N
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

top_n = get_top_n(predictions, n=5)

# Show recommendations for a few users
for uid, user_recs in list(top_n.items())[:3]:
    print(f"\nTop 5 recommendations for User {uid}:")
    for iid, rating in user_recs:
        print(f"Movie ID {iid} with predicted rating {round(rating, 2)}")


Top 5 recommendations for User 907:
Movie ID 143 with predicted rating 5
Movie ID 173 with predicted rating 5
Movie ID 172 with predicted rating 5
Movie ID 647 with predicted rating 5
Movie ID 185 with predicted rating 5

Top 5 recommendations for User 371:
Movie ID 210 with predicted rating 4.33
Movie ID 186 with predicted rating 4.26
Movie ID 97 with predicted rating 4.16
Movie ID 746 with predicted rating 4.11
Movie ID 431 with predicted rating 3.94

Top 5 recommendations for User 218:
Movie ID 12 with predicted rating 3.97
Movie ID 654 with predicted rating 3.8
Movie ID 209 with predicted rating 3.76
Movie ID 789 with predicted rating 3.62
Movie ID 42 with predicted rating 3.44
